In [172]:
import six
from itertools import product

In [173]:
DATA = """
.#.
..#
###
""".strip()

In [189]:
class HyperSpace:
    
    """
    Utility to solve AoC's 17th puzzle of 2020; it stores
    a representation of a D-dimensional space populated by
    active and inactive cubes, which at any time step update
    their state based on a set of given rules.
    """
    
    def __init__(self, layer, numberOfDimensions=3):
        
        """
        Initialises the hyperspace with a single
        2-dimensional slice.
        
        :param layer: list(list(str,))
        :param numberOfDimensions: int
        """
        
        self.numberOfDimensions = numberOfDimensions  
        
        # The 'active' set contains the coordinates of all active
        # hypercubes; this is because it is assumed that everything
        # is not activate by default aside from the given active
        # inputs and those that become active as time progresses;
        
        self.active = {tuple([x, y] + [0] * (numberOfDimensions - 2)) for y, row in enumerate(layer.splitlines()) for x, value in enumerate(row) if value == '#'}
                
    def neighbours(self, *coordinates):
        
        """
        Yields the coordinates of all the neighbours of the given coordinate.
        
        :param coordinates: (int,)
        :yield: neighbourCoordinates:tuple(int,), isActive:bool
        """
        
        # To find the neighbours, the (Cartesian) product of the
        # unit displacement in all dimension is considered, but
        # excluding the null displacement as it would consider
        # the given coordinates as a neeighbour;
        
        for displacements in product(*[(-1, 0, +1) for _ in range(self.numberOfDimensions)]):
            if all(displacement == 0 for displacement in displacements): continue
            neighbourCoordinates = tuple([coordinate + displacement for coordinate, displacement in zip(coordinates, displacements)])
            yield neighbourCoordinates
            
    def takeSteps(self, n):
        """
        Performs a timestep in the hyperspace and updates the
        state of every cube based on the given rules.
        """
        for _ in range(n): self._step()
        return self
    
    def _step(self):
        
        # First, all the coordinates (out of the infinitely
        # available locations) which might need updating
        # are found; these are all the active coordinates
        # with the addition (set union) of all their neighbours;
        # this is because, as per the given rules, an inactive
        # cube (which is the default in the hyperspace) will
        # only update if it has exactly three active neighbours,
        # hence only direct neighbours of active known cubes are
        # considered.
        
        toInspect = set(self.active).union(neighbourCoordinates for coordinates in self.active for neighbourCoordinates in self.neighbours(*coordinates))
            
        # For all known locations, determine whether their
        # values needs updating and store the result in a
        # temporary set (this allows updating 'simultaneously');
        
        toActivate, toDeactivate = set(), set()
        for coordinates in toInspect:
            isActive = coordinates in self.active
            neighbours = set(self.neighbours(*coordinates))
            numberOfActiveNeighbours = sum(isActive for _, isActive in neighbours)
            if isActive and numberOfActiveNeighbours not in (2, 3): toDeactivate.add(coordinates)
            elif not isActive and numberOfActiveNeighbours == 3: toActivate.add(coordinates)

        self.active.update(toActivate)
        self.active = self.active.difference(toDeactivate)
            
    def count(self):
        """
        Retrieves how many active cubes there are in the hyperspace.
        
        :return: int
        """
        return len(self.active)

In [190]:
HyperSpace(DATA, numberOfDimensions=3).takeSteps(6).count()

112

In [183]:
HyperSpace(DATA, numberOfDimensions=4).takeSteps(6).count()

848

In [184]:
a = [0] * 4

In [186]:
a[0] = 1
a

[1, 0, 0, 0]